In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import librairies 📚

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
plt.style.use('ggplot')

import cufflinks as cf
import plotly.express as px
import plotly.offline as py
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.graph_objs as go

try:
    import apyori
except:
    !pip install apyori

from apyori import apriori

## Reading Data 📝

In [ ]:
df = pd.read_csv("../input/groceries-dataset/Groceries_dataset.csv")
df.head()

In [ ]:
print("We have :", df.shape[0], "items in the Dataset")
print(f"And null values :\n{df.isnull().sum()}")

In [ ]:
print(f"And nan values :\n{df.isna().sum()} ")

In [ ]:
print(f'There are : {len(df["itemDescription"].unique())} unique items')

In [ ]:
df["Year"] = df["Date"].str.split("-").str[-1]
df["Months_Year"] = df["Date"].str.split("-").str[1] + "-" + df["Date"].str.split("-").str[-1] 
df.head()

In [ ]:
df["Months_Year"].unique()

## Data Exploration 📊

In [ ]:
#Graph : Months/Year by count
fig = px.bar(df["Months_Year"].value_counts(ascending=False), orientation="v", color=df["Months_Year"].value_counts(ascending=False), color_continuous_scale=px.colors.sequential.Plasma, 
             log_x=False, labels={'value':'Count', 
                                'index':'Date',
                                 'color':'None'
                                })

fig.update_layout(
    font_color="black",
    title_font_color="red",
    legend_title_font_color="green",
    title_text="Date by count"
)

fig.show()

We buy more in october and conversely, in march we buy less. (in this case)

In [ ]:
#Graph : Item by count
fig = px.bar(df["itemDescription"].value_counts()[:20], orientation="v", color=df["itemDescription"].value_counts()[:20], color_continuous_scale=px.colors.sequential.Plasma, 
             log_x=False, labels={'value':'Count', 
                                'index':'Item',
                                 'color':'None'
                                })

fig.update_layout(
    font_color="black",
    title_font_color="red",
    legend_title_font_color="green",
    title_text="Item by count"
)

fig.show()

In this dataset, we buy a lot of milk.

## Feature Engineering 🏷️

In [ ]:
products = df["itemDescription"].unique()

In [ ]:
one_hot = pd.get_dummies(df['itemDescription'])

df.drop(['itemDescription'], inplace=True, axis=1)
df = df.join(one_hot)

df.head()

In [ ]:
records = df.groupby(["Member_number","Date"])[products[:]].sum()
records = records.reset_index()[products]

records

To create an apriory model we have to change the 0 values into the name of the product :
(I also saw that's better if we create transactions to group transactions in the same day)

In [ ]:
def get_product_names(x):
    for product in products:
        if x[product] != 0:
            x[product] = product
    return x

records = records.apply(get_product_names, axis=1)
records.head()

print(f"Total transactions: {len(records)}")

In [ ]:
# Removing zeros in DataFrame
x = records.values
x = [sub[~(sub == 0)].tolist() for sub in x if sub[sub != 0].tolist()]
transactions = x

## Modelling 🟩

In [ ]:
association_rules = apriori(transactions,min_support=0.00030, min_confidance=0.01, min_lift=3, min_length=2, target="rules")
association_results = list(association_rules)

In [ ]:
print(association_results[0])

In [ ]:
for item in association_results:

    pair = item[0] 
    items = [x for x in pair]
    
    print("Rule : ", items[0], " -> " + items[1])
    print("Support : ", str(item[1]))
    print("Confidence : ",str(item[2][0][2]))
    print("Lift : ", str(item[2][0][3]))
    
    print("=====================================")